In [ ]:
# Install required libraries without upgrading pandas
!pip install yfinance plotly -q
!pip install pandas_ta --no-deps -q

In [ ]:
# Install required libraries
!pip install yfinance plotly -q
!pip install pandas_ta --no-deps -q

import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Configuration
symbol = 'AAPL'
initial_capital = 100000
risk_per_trade = 0.015
reward_risk_ratio = 2.0

def prepare_dataframe(data):
    """Clean and prepare DataFrame"""
    df = data.reset_index()

    if isinstance(df.columns, pd.MultiIndex):
        new_columns = []
        for col in df.columns:
            if isinstance(col, tuple):
                if col[0] in ['Date', 'Datetime'] or col[0] == '':
                    new_columns.append('Date')
                else:
                    new_columns.append(col[0])
            else:
                new_columns.append(col)
        df.columns = new_columns

    if 'Datetime' in df.columns:
        df.rename(columns={'Datetime': 'Date'}, inplace=True)

    return df

def add_technical_indicators(df):
    """Add technical indicators for the strategy"""
    # EMAs for trend filter
    df['EMA_20'] = ta.ema(df['Close'], length=20)
    df['EMA_50'] = ta.ema(df['Close'], length=50)

    # RSI for additional filter
    df['RSI'] = ta.rsi(df['Close'], length=14)

    # Bollinger Bands - core of mean reversion
    bb = ta.bbands(df['Close'], length=20, std=2)
    if bb is not None and isinstance(bb, pd.DataFrame):
        bb_cols = bb.columns.tolist()
        lower_col = [col for col in bb_cols if 'BBL' in col][0] if any('BBL' in col for col in bb_cols) else bb_cols[0]
        mid_col = [col for col in bb_cols if 'BBM' in col][0] if any('BBM' in col for col in bb_cols) else bb_cols[1]
        upper_col = [col for col in bb_cols if 'BBU' in col][0] if any('BBU' in col for col in bb_cols) else bb_cols[2]
        df['BB_Lower'] = bb[lower_col]
        df['BB_Mid'] = bb[mid_col]
        df['BB_Upper'] = bb[upper_col]
    else:
        df['BB_Mid'] = df['Close'].rolling(window=20).mean()
        bb_std = df['Close'].rolling(window=20).std()
        df['BB_Upper'] = df['BB_Mid'] + (bb_std * 2)
        df['BB_Lower'] = df['BB_Mid'] - (bb_std * 2)

    # ATR for position sizing
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
    if df['ATR'].isna().all():
        df['TR'] = np.maximum(df['High'] - df['Low'],
                             np.maximum(abs(df['High'] - df['Close'].shift(1)),
                                       abs(df['Low'] - df['Close'].shift(1))))
        df['ATR'] = df['TR'].rolling(window=14).mean()
        df.drop('TR', axis=1, inplace=True)

    # BB position for mean reversion
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    df['BB_Position'] = df['BB_Position'].fillna(0.5)

    return df

def detect_candlestick_patterns(df):
    """Detect candlestick reversal patterns"""

    # Calculate candle components
    df['Body_Size'] = abs(df['Close'] - df['Open'])
    df['Upper_Shadow'] = df['High'] - np.maximum(df['Close'], df['Open'])
    df['Lower_Shadow'] = np.minimum(df['Close'], df['Open']) - df['Low']
    df['Candle_Range'] = df['High'] - df['Low']

    # Avoid division by zero
    df['Body_Size'] = df['Body_Size'].replace(0, 0.001)
    df['Candle_Range'] = df['Candle_Range'].replace(0, 0.001)

    # HAMMER (Bullish reversal)
    df['Hammer'] = (
        (df['Lower_Shadow'] > 2 * df['Body_Size']) &
        (df['Upper_Shadow'] < 0.5 * df['Body_Size']) &
        (df['Body_Size'] > 0) &
        (df['Body_Size'] < 0.3 * df['Candle_Range'])
    )

    # INVERTED HAMMER (Bullish reversal)
    df['Inverted_Hammer'] = (
        (df['Upper_Shadow'] > 2 * df['Body_Size']) &
        (df['Lower_Shadow'] < 0.5 * df['Body_Size']) &
        (df['Body_Size'] > 0) &
        (df['Body_Size'] < 0.3 * df['Candle_Range'])
    )

    # SHOOTING STAR (Bearish reversal)
    df['Shooting_Star'] = (
        (df['Upper_Shadow'] > 2 * df['Body_Size']) &
        (df['Lower_Shadow'] < 0.5 * df['Body_Size']) &
        (df['Body_Size'] > 0) &
        (df['Body_Size'] < 0.3 * df['Candle_Range'])
    )

    # HANGING MAN (Bearish reversal)
    df['Hanging_Man'] = (
        (df['Lower_Shadow'] > 2 * df['Body_Size']) &
        (df['Upper_Shadow'] < 0.5 * df['Body_Size']) &
        (df['Body_Size'] > 0) &
        (df['Body_Size'] < 0.3 * df['Candle_Range'])
    )

    # DOJI (Indecision/reversal)
    df['Doji'] = (df['Body_Size'] < 0.1 * df['Candle_Range'])

    # ENGULFING PATTERNS
    df['Bullish_Engulfing'] = False
    df['Bearish_Engulfing'] = False

    for i in range(1, len(df)):
        # Bullish Engulfing
        prev_bearish = df['Close'].iloc[i-1] < df['Open'].iloc[i-1]
        curr_bullish = df['Close'].iloc[i] > df['Open'].iloc[i]
        curr_engulfs = (df['Open'].iloc[i] < df['Close'].iloc[i-1]) and (df['Close'].iloc[i] > df['Open'].iloc[i-1])
        df.loc[i, 'Bullish_Engulfing'] = prev_bearish and curr_bullish and curr_engulfs

        # Bearish Engulfing
        prev_bullish = df['Close'].iloc[i-1] > df['Open'].iloc[i-1]
        curr_bearish = df['Close'].iloc[i] < df['Open'].iloc[i]
        curr_engulfs = (df['Open'].iloc[i] > df['Close'].iloc[i-1]) and (df['Close'].iloc[i] < df['Open'].iloc[i-1])
        df.loc[i, 'Bearish_Engulfing'] = prev_bullish and curr_bearish and curr_engulfs

    # Combine patterns
    df['Bullish_Pattern'] = (
        df['Hammer'] | df['Inverted_Hammer'] |
        df['Bullish_Engulfing'] |
        (df['Doji'] & (df['RSI'] < 40))
    )

    df['Bearish_Pattern'] = (
        df['Shooting_Star'] | df['Hanging_Man'] |
        df['Bearish_Engulfing'] |
        (df['Doji'] & (df['RSI'] > 60))
    )

    return df

def candlestick_bb_strategy(df):
    """Candlestick + Bollinger Band Mean Reversion Strategy"""
    df = df.copy()
    df = detect_candlestick_patterns(df)

    # Near Bollinger Band extremes
    df['Near_BB_Lower'] = df['BB_Position'] < 0.15
    df['Near_BB_Upper'] = df['BB_Position'] > 0.85

    # Price momentum confirmation
    df['Price_Turning_Up'] = df['Close'] > df['Close'].shift(1)
    df['Price_Turning_Down'] = df['Close'] < df['Close'].shift(1)

    # Buy signals: Bullish pattern at BB lower extreme
    df['Buy_Signal'] = (
        df['Bullish_Pattern'] &
        df['Near_BB_Lower'] &
        df['Price_Turning_Up'] &
        (df['RSI'] < 40)
    )

    # Sell signals: Bearish pattern at BB upper extreme
    df['Sell_Signal'] = (
        df['Bearish_Pattern'] &
        df['Near_BB_Upper'] &
        df['Price_Turning_Down'] &
        (df['RSI'] > 60)
    )

    return df

def backtest_strategy(df):
    """Backtest the candlestick + BB strategy"""
    capital = initial_capital
    position = 0
    trades = []
    signal_log = []  # Log all signals
    equity = [initial_capital]

    entry_price = 0
    stop_loss = 0
    take_profit = 0

    for i in range(50, len(df)):
        current_price = df['Close'].iloc[i]
        current_date = df['Date'].iloc[i]
        current_atr = df['ATR'].iloc[i] if not pd.isna(df['ATR'].iloc[i]) else current_price * 0.02

        # Log signals
        if df['Buy_Signal'].iloc[i]:
            signal_log.append({
                'Date': current_date,
                'Signal_Type': 'Buy',
                'Price': current_price,
                'Executed': False
            })
        if df['Sell_Signal'].iloc[i]:
            signal_log.append({
                'Date': current_date,
                'Signal_Type': 'Sell',
                'Price': current_price,
                'Executed': False
            })

        # Update equity
        if position != 0:
            equity.append(capital + position * current_price)
        else:
            equity.append(capital)

        # Check exits
        if position > 0:  # Long position
            if current_price <= stop_loss or current_price >= take_profit:
                exit_price = stop_loss if current_price <= stop_loss else take_profit
                pnl = position * (exit_price - entry_price)
                capital += position * exit_price
                trades.append({
                    'Exit_Date': current_date,
                    'Exit_Price': exit_price,
                    'P&L': pnl,
                    'Type': 'Stop Loss' if current_price <= stop_loss else 'Take Profit'
                })
                position = 0

        elif position < 0:  # Short position
            if current_price >= stop_loss or current_price <= take_profit:
                exit_price = stop_loss if current_price >= stop_loss else take_profit
                pnl = position * (exit_price - entry_price)
                capital += position * exit_price
                trades.append({
                    'Exit_Date': current_date,
                    'Exit_Price': exit_price,
                    'P&L': pnl,
                    'Type': 'Stop Loss' if current_price >= stop_loss else 'Take Profit'
                })
                position = 0

        # Check entries
        if position == 0 and df['Buy_Signal'].iloc[i]:  # Only buy if no position
            risk_amount = capital * risk_per_trade
            pattern_stop_distance = max(current_price - df['Low'].iloc[i], current_atr * 0.8)

            if pattern_stop_distance > 0:
                position_size = risk_amount / pattern_stop_distance
                position = position_size
                entry_price = current_price
                stop_loss = current_price - pattern_stop_distance
                take_profit = current_price + (pattern_stop_distance * reward_risk_ratio)
                capital -= position * entry_price

                trades.append({
                    'Entry_Date': current_date,
                    'Entry_Type': 'Buy',
                    'Entry_Price': entry_price,
                    'Stop': stop_loss,
                    'Target': take_profit
                })
                # Mark signal as executed
                signal_log[-1]['Executed'] = True if signal_log and signal_log[-1]['Signal_Type'] == 'Buy' and signal_log[-1]['Date'] == current_date else False

        elif position > 0 and df['Sell_Signal'].iloc[i]:  # Only sell if holding long
            risk_amount = capital * risk_per_trade
            pattern_stop_distance = max(df['High'].iloc[i] - current_price, current_atr * 0.8)

            if pattern_stop_distance > 0:
                # Close long position
                capital += position * current_price
                trades.append({
                    'Exit_Date': current_date,
                    'Exit_Price': current_price,
                    'P&L': position * (current_price - entry_price),
                    'Type': 'Sell Signal'
                })
                # Open short position
                position_size = risk_amount / pattern_stop_distance
                position = -position_size
                entry_price = current_price
                stop_loss = current_price + pattern_stop_distance
                take_profit = current_price - (pattern_stop_distance * reward_risk_ratio)
                capital += position_size * entry_price

                trades.append({
                    'Entry_Date': current_date,
                    'Entry_Type': 'Sell',
                    'Entry_Price': entry_price,
                    'Stop': stop_loss,
                    'Target': take_profit
                })
                # Mark signal as executed
                signal_log[-1]['Executed'] = True if signal_log and signal_log[-1]['Signal_Type'] == 'Sell' and signal_log[-1]['Date'] == current_date else False

    # Close final position
    if position != 0:
        final_price = df['Close'].iloc[-1]
        pnl = position * (final_price - entry_price)
        capital += position * final_price
        trades.append({
            'Exit_Date': df['Date'].iloc[-1],
            'Exit_Price': final_price,
            'P&L': pnl,
            'Type': 'Final Close'
        })

    # Calculate metrics
    total_return = (capital - initial_capital) / initial_capital * 100
    num_trades = len([t for t in trades if 'Entry_Type' in t])

    pnl_trades = [t['P&L'] for t in trades if 'P&L' in t]
    winning_trades = [p for p in pnl_trades if p > 0]
    losing_trades = [p for p in pnl_trades if p <= 0]

    win_rate = len(winning_trades) / len(pnl_trades) * 100 if pnl_trades else 0
    avg_win = np.mean(winning_trades) if winning_trades else 0
    avg_loss = np.mean(losing_trades) if losing_trades else 0
    profit_factor = abs(sum(winning_trades) / sum(losing_trades)) if losing_trades else float('inf')

    # Calculate max drawdown
    equity_series = pd.Series(equity)
    peak = equity_series.expanding().max()
    drawdown = (equity_series - peak) / peak * 100
    max_drawdown = drawdown.min()

    # Ensure equity matches dataframe length
    while len(equity) < len(df):
        equity.append(equity[-1])

    return {
        'final_capital': capital,
        'total_return': total_return,
        'num_trades': num_trades,
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'profit_factor': profit_factor,
        'max_drawdown': max_drawdown,
        'equity_curve': equity[:len(df)],
        'trades': trades,
        'signal_log': signal_log,
        'dataframe': df
    }

# Fetch extended data (5 years for longer backtest)
start_date = '2020-01-01'  # 5 years instead of 2
end_date = '2025-01-01'

print("Fetching 5-year data for extended backtest...")
data = yf.download(symbol, start=start_date, end=end_date, interval='1d', auto_adjust=True, progress=False)
df = prepare_dataframe(data)

print(f"Data: {len(df)} bars from {df['Date'].iloc[0]} to {df['Date'].iloc[-1]}")

# Add indicators and run strategy
df = add_technical_indicators(df)
df = candlestick_bb_strategy(df)
results = backtest_strategy(df)

# Display results
print("\n" + "="*60)
print("CANDLESTICK + BOLLINGER BAND STRATEGY RESULTS (5 YEARS)")
print("="*60)
print(f"Initial Capital:     ${initial_capital:,.2f}")
print(f"Final Capital:       ${results['final_capital']:,.2f}")
print(f"Total Return:        {results['total_return']:.2f}%")
print(f"Annualized Return:   {results['total_return']/5:.2f}%")
print(f"Number of Trades:    {results['num_trades']} ({results['num_trades']/5:.1f} per year)")
print(f"Win Rate:            {results['win_rate']:.1f}%")
print(f"Average Win:         ${results['avg_win']:,.2f}")
print(f"Average Loss:        ${results['avg_loss']:,.2f}")
print(f"Profit Factor:       {results['profit_factor']:.2f}" if results['profit_factor'] != float('inf') else "Profit Factor:       ∞")
print(f"Max Drawdown:        {results['max_drawdown']:.1f}%")

# Trade summary
entry_trades = [t for t in results['trades'] if 'Entry_Type' in t]
if entry_trades:
    print(f"\nTRADE SUMMARY:")
    print(f"Buy trades:  {len([t for t in entry_trades if t['Entry_Type'] == 'Buy'])}")
    print(f"Sell trades: {len([t for t in entry_trades if t['Entry_Type'] == 'Sell'])}")

# Show all trades
print(f"\nALL TRADES:")
for i, trade in enumerate(entry_trades, 1):
    entry_date = trade['Entry_Date'].strftime('%Y-%m-%d')
    trade_type = trade['Entry_Type']
    entry_price = trade['Entry_Price']

    # Find corresponding exit
    exit_trade = None
    for exit_t in results['trades']:
        if 'P&L' in exit_t and exit_t['Exit_Date'] > trade['Entry_Date']:
            exit_trade = exit_t
            break

    if exit_trade:
        exit_date = exit_trade['Exit_Date'].strftime('%Y-%m-%d')
        exit_price = exit_trade['Exit_Price']
        pnl = exit_trade['P&L']
        days_held = (exit_trade['Exit_Date'] - trade['Entry_Date']).days
        print(f"{i:2d}. {entry_date} {trade_type:4} @${entry_price:6.2f} → {exit_date} @${exit_price:6.2f} | {days_held:3d}d | ${pnl:+7.0f}")

# Show all signals
print(f"\nALL SIGNALS:")
for i, signal in enumerate(results['signal_log'], 1):
    signal_date = signal['Date'].strftime('%Y-%m-%d')
    signal_type = signal['Signal_Type']
    signal_price = signal['Price']
    executed = 'Yes' if signal['Executed'] else 'No'
    print(f"{i:2d}. {signal_date} {signal_type:4} @${signal_price:6.2f} | Executed: {executed}")

# Visualization
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=['Price Action & Signals', 'RSI & Bollinger Band Position', 'Portfolio Equity'],
    row_heights=[0.5, 0.25, 0.25],
    vertical_spacing=0.05
)

# Candlestick chart
fig.add_trace(go.Candlestick(
    x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'],
    name='Price'
), row=1, col=1)

# Bollinger Bands
fig.add_trace(go.Scatter(x=df['Date'], y=df['BB_Upper'], name='BB Upper',
                        line=dict(color='gray', dash='dash'), opacity=0.7), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=df['BB_Lower'], name='BB Lower',
                        line=dict(color='gray', dash='dash'), opacity=0.7,
                        fill='tonexty', fillcolor='rgba(128,128,128,0.1)'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Date'], y=df['BB_Mid'], name='BB Mid',
                        line=dict(color='blue', width=1)), row=1, col=1)

# Trading signals
buy_signals = df[df['Buy_Signal'] == True]
sell_signals = df[df['Sell_Signal'] == True]

if len(buy_signals) > 0:
    fig.add_trace(go.Scatter(x=buy_signals['Date'], y=buy_signals['Close'],
                            mode='markers', marker=dict(symbol='triangle-up', size=12, color='green'),
                            name='Buy Signals'), row=1, col=1)

if len(sell_signals) > 0:
    fig.add_trace(go.Scatter(x=sell_signals['Date'], y=sell_signals['Close'],
                            mode='markers', marker=dict(symbol='triangle-down', size=12, color='red'),
                            name='Sell Signals'), row=1, col=1)

# RSI
fig.add_trace(go.Scatter(x=df['Date'], y=df['RSI'], name='RSI', line=dict(color='purple')), row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, opacity=0.7)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, opacity=0.7)

# BB Position
fig.add_trace(go.Scatter(x=df['Date'], y=df['BB_Position'], name='BB Position',
                        line=dict(color='orange')), row=2, col=1)
fig.add_hline(y=0.15, line_dash="dash", line_color="green", row=2, col=1, opacity=0.5)
fig.add_hline(y=0.85, line_dash="dash", line_color="red", row=2, col=1, opacity=0.5)

# Equity curve
fig.add_trace(go.Scatter(x=df['Date'], y=results['equity_curve'], name='Portfolio Value',
                        line=dict(color='darkgreen', width=3)), row=3, col=1)
fig.add_hline(y=initial_capital, line_dash="dash", line_color="black", row=3, col=1, opacity=0.5)

# Layout
fig.update_layout(
    height=1000,
    title=f'{symbol} - Candlestick + Bollinger Band Mean Reversion Strategy (5 Year Backtest)',
    xaxis_rangeslider_visible=False,
    showlegend=True
)

fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="RSI / BB Position", row=2, col=1, range=[0, 100])
fig.update_yaxes(title_text="Portfolio Value ($)", row=3, col=1)

fig.show()

print(f"\n" + "="*60)
print("STRATEGY SUMMARY")
print("="*60)
print("✅ Mean reversion strategy using candlestick patterns at BB extremes")
print("✅ Natural stop losses based on candle highs/lows")
print("✅ 2:1 reward-to-risk ratio for profitable trades")
print("✅ RSI filter prevents trades in extreme momentum")
print("✅ Extended 5-year backtest for robust validation")
print("✅ Modified to only allow selling when holding a long position")
print("✅ Modified to prevent new buys until existing position is sold")
print("✅ Added logging of all buy and sell signals with execution status")

Fetching 5-year data for extended backtest...
Data: 1258 bars from 2020-01-02 00:00:00 to 2024-12-31 00:00:00

CANDLESTICK + BOLLINGER BAND STRATEGY RESULTS (5 YEARS)
Initial Capital:     $100,000.00
Final Capital:       $101,455.00
Total Return:        1.46%
Annualized Return:   0.29%
Number of Trades:    2 (0.4 per year)
Win Rate:            50.0%
Average Win:         $3,000.00
Average Loss:        $-1,545.00
Profit Factor:       1.94
Max Drawdown:        -2.3%

TRADE SUMMARY:
Buy trades:  2
Sell trades: 0

ALL TRADES:
 1. 2022-10-10 Buy  @$138.24 → 2022-10-24 @$145.52 |  14d | $  +3000
 2. 2023-08-08 Buy  @$177.82 → 2023-08-16 @$175.14 |   8d | $  -1545

ALL SIGNALS:
 1. 2020-08-25 Sell @$121.42 | Executed: No
 2. 2020-08-27 Sell @$121.60 | Executed: No
 3. 2020-09-02 Sell @$127.82 | Executed: No
 4. 2020-12-23 Sell @$127.61 | Executed: No
 5. 2021-04-14 Sell @$128.84 | Executed: No
 6. 2021-06-23 Sell @$130.69 | Executed: No
 7. 2021-08-17 Sell @$147.03 | Executed: No
 8. 2022-07-2


STRATEGY SUMMARY
✅ Mean reversion strategy using candlestick patterns at BB extremes
✅ Natural stop losses based on candle highs/lows
✅ 2:1 reward-to-risk ratio for profitable trades
✅ RSI filter prevents trades in extreme momentum
✅ Extended 5-year backtest for robust validation
✅ Modified to only allow selling when holding a long position
✅ Modified to prevent new buys until existing position is sold
✅ Added logging of all buy and sell signals with execution status
